Import Required Packages

In [5]:
import glob
import numpy as np
import pandas as pd
import cv2 as cv
import os
import glob
import random
from PIL import Image


from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
from sklearn.svm import SVC

from utils import *

In [7]:
classN_img_path = "Data/train/normal"
classA_img_path = "Data/train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib"
classL_img_path = "Data/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa"
classS_img_path = "Data/train/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa"

image_paths = {"normal": classN_img_path, 
               "adenocarcinoma": classA_img_path,
               "large_cell_carcinoma": classL_img_path, 
               "squamous_cell_carcinoma": classS_img_path}

In [31]:
def resize_images(folder_path, target_size):
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            img = Image.open(os.path.join(folder_path, filename))
            img = img.resize(target_size, Image.LANCZOS)
            base_filename, file_extension = os.path.splitext(filename)
            img.save(os.path.join(folder_path, f'{base_filename}_resized{file_extension}'))

In [48]:
def align_images(image_files):
    orb = cv2.ORB_create()
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    reference_image_path = random.choice(image_files)
    img1 = cv2.imread(reference_image_path, 0)
    kp1, des1 = orb.detectAndCompute(img1, None)

    for image_path in image_files:
        if image_path != reference_image_path:
            img2 = cv2.imread(image_path, 0)

            kp2, des2 = orb.detectAndCompute(img2, None)
            matches = bf.match(des1, des2)
            matches = sorted(matches, key=lambda x: x.distance)

            points1 = np.zeros((len(matches), 2), dtype=np.float32)
            points2 = np.zeros((len(matches), 2), dtype=np.float32)

            for i, match in enumerate(matches):
                points1[i, :] = kp1[match.queryIdx].pt
                points2[i, :] = kp2[match.trainIdx].pt

            h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

            img2_aligned = cv2.warpPerspective(img2, h, (img1.shape[1], img1.shape[0]))

            aligned_image_path = os.path.splitext(image_path)[0] + '_aligned' + os.path.splitext(image_path)[1]
            cv2.imwrite(aligned_image_path, img2_aligned)

def align_resized_images(folder_path):
    # Get all image files in the folder
    image_files = glob.glob(os.path.join(folder_path, '*resized*'))
    

    # Align images
    align_images(image_files)


In [62]:
def compute_mean_image(image_files):
    img_sum = None
    
    for image_file in image_files:
        img = cv2.imread(image_file).astype(np.float32)
        if img_sum is None:
            img_sum = img
        else:
            img_sum += img

    mean_img = img_sum / len(image_files)
    mean_img = mean_img.astype(np.uint8)

    return mean_img

In [71]:
def align_to_mean_image(image_files, mean_img):
    orb = cv2.ORB_create()
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    kp1, des1 = orb.detectAndCompute(mean_img, None)

    for image_path in image_files:
        if not '_aligned' in image_path and '_resized' in image_path:
            img2 = cv2.imread(image_path, 0)

            kp2, des2 = orb.detectAndCompute(img2, None)
            matches = bf.match(des1, des2)

            # Ensure there are enough matches
            if len(matches) < 4:
                print(f"Not enough matches found in {image_path}")
                continue

            matches = sorted(matches, key=lambda x: x.distance)

            points1 = np.zeros((len(matches), 2), dtype=np.float32)
            points2 = np.zeros((len(matches), 2), dtype=np.float32)

            for i, match in enumerate(matches):
                points1[i, :] = kp1[match.queryIdx].pt
                points2[i, :] = kp2[match.trainIdx].pt

            h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
            
            if h is None:
                print(f"Could not find a valid homography for image {image_path}.")
                continue

            img2_aligned = cv2.warpPerspective(img2, h, (mean_img.shape[1], mean_img.shape[0]))

            # Overwrite the aligned image if it exists
            aligned_image_path = image_path.rsplit('_', 1)[0] + '_aligned' + os.path.splitext(image_path)[1]
            cv2.imwrite(aligned_image_path, img2_aligned)


In [72]:
def process_folder(folder_path, max_iterations=10, convergence_threshold=1e-6):
    # Get all image files in the folder
    image_files = glob.glob(os.path.join(folder_path, '*resized*'))

    # Compute initial mean image
    mean_img = compute_mean_image(image_files)

    for _ in range(max_iterations):
        # Align images to current mean image
        align_to_mean_image(image_files, mean_img)

        # Compute new mean image
        new_mean_img = compute_mean_image([f.rsplit('_', 1)[0] + '_aligned' + os.path.splitext(f)[1] for f in image_files])

        # Compute the difference between the new and old mean image
        diff = np.mean(np.abs(new_mean_img - mean_img))

        # Update the mean image
        mean_img = new_mean_img

        # If the difference is below the convergence threshold, stop iterating
        if diff < convergence_threshold:
            break

    # Save the final mean image
    cv2.imwrite(os.path.join(folder_path, 'mean_image.jpg'), mean_img)

In [73]:
def remove_altered_images(folder_path):
    for filename in os.listdir(folder_path):
        if 'resized' in filename or 'aligned' in filename or 'mean_image' in filename:
            os.remove(os.path.join(folder_path, filename))

In [74]:
for label, image_path in image_paths.items():
    remove_resized_images(image_path)
    resize_images(image_path, (256, 256))
    align_resized_images(image_path)
    process_folder(image_path)
    
    

[ WARN:0@94907.229] global loadsave.cpp:248 findDecoder imread_('Data/train/normal/5 - Copy - Copy_aligned_aligned.png'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'astype'

In [ ]:
def deduplicate_images(folder_path, rmse_threshold=7.0, target_size=(256, 256)):
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif'))]
    deduplicated_images = []

    for i, image_file in enumerate(image_files):
        image1_path = os.path.join(folder_path, image_file)
        image1 = Image.open(image1_path)

        # Convert the image to grayscale
        image1 = image1.convert('L')

        # Resize the image to the target size
        image1 = resize_image(image1, target_size)

        is_duplicate = False
        for dedup_image in deduplicated_images:
            image2 = dedup_image[0]
            rmse = compute_rmse(image1, image2)
            if rmse < rmse_threshold:
                is_duplicate = True
                break

        if not is_duplicate:
            deduplicated_images.append((image1, image_file))
    
    deduplicated_images.sort(key=lambda x: x[1])
    
    return [image_file for _, image_file in deduplicated_images]


In [ ]:

def display_img_colorbar(img):
  # display the points
  fig, ax = plt.subplots(figsize=(15, 10), nrows=2, ncols=2)
  im_ax = plt.imshow(img, cmap='gray')
  # create an axes on the right side of ax. The width of cax will be 5%
  # of ax and the padding between cax and ax will be fixed at 0.05 inch.
  divider = make_axes_locatable(ax)
  cax = divider.append_axes("right", size="5%", pad=0.05)
  plt.colorbar(im_ax, cax=cax)
  plt.show()

def plot_imgs(imN, imA, imL, imS):
    # display the points
    fig, ax = plt.subplots(figsize=(15, 10), nrows=2, ncols=2)
    ax[0][0].imshow(imN, cmap='gray')
    ax[0][0].set_title("Normal (N)")

    ax[0][1].imshow(imA, cmap='gray')
    ax[0][1].set_title("Adenocarcinoma (A)")

    ax[1][0].imshow(imL, cmap='gray')
    ax[1][0].set_title("Large cell carcinoma (L)")

    ax[1][1].imshow(imS, cmap='gray')
    ax[1][1].set_title("Squamous cell carcinoma (S)")
    plt.show()

In [ ]:
def generate_edges(img):
    # extract the features from the image

    # convert to grayscale
    if np.max(img)>1:
        img = img.astype(np.float32)/255.0
    im_gray = np.mean(img, axis=2)

    # compute edges of the image
    sobelx = cv.Sobel(im_gray, cv.CV_32F, 1, 0, ksize=21) # Find x and y gradients
    sobely = cv.Sobel(im_gray, cv.CV_32F, 0, 1, ksize=21)
    magnitude = np.sqrt(sobelx**2.0 + sobely**2.0)
    magnitude = magnitude / np.max(magnitude) # normalize

    # threshold the image and get the interesting points
    im_threshold = cv.Canny(image=(magnitude * 255).astype(np.uint8), threshold1=0, threshold2=100) # Canny Edge
    im_threshold = im_threshold / np.max(im_threshold) # normalize

    return magnitude, im_threshold

In [ ]:
edgesN, canny_edgesN = generate_edges(im_N)
edgesA, canny_edgesA = generate_edges(im_A)
edgesL, canny_edgesL = generate_edges(im_L)
edgesS, canny_edgesS = generate_edges(im_S)

In [ ]:
plot_imgs(im_N, im_A, im_L, im_S)

In [ ]:
plot_imgs(edgesN, edgesA, edgesL, edgesS)

In [ ]:
plot_imgs(canny_edgesN, canny_edgesA, canny_edgesL, canny_edgesS)


Feature Extraction

In [ ]:
class_mappings = {
    0: "normal",
    1: "adenocarcinoma",
    2: "large.cell.carcinoma",
    3: "squamous.cell.carcinoma"
}

In [ ]:

train_path = "/content/drive/MyDrive/W281/Final Project/Data_Cropped_and_Resized/train"

train_imgs, train_sobel_edges, train_labels = extract_features(train_path, detect_edges_sobel, class_mappings)
_, train_hounsfield_edges, _ = extract_features(train_path, apply_hounsfield_units, class_mappings)

In [ ]:
valid_path = "/content/drive/MyDrive/W281/Final Project/Data_Cropped_and_Resized/valid"

valid_imgs, valid_sobel_edges, valid_labels = extract_features(valid_path, detect_edges_sobel, class_mappings)
_, valid_hounsfield_edges, _ = extract_features(valid_path, apply_hounsfield_units, class_mappings)

In [ ]:
plot_features(train_imgs, train_sobel_edges, train_labels, 0, 'Sobel Edge', class_mappings)


In [ ]:
plot_features(train_imgs, train_hounsfield_edges, train_labels, 0, "Hounsfield Unit", class_mappings)


In [ ]:
train_path = "/content/drive/MyDrive/W281/Final Project/Data/train"
val_path = "/content/drive/MyDrive/W281/Final Project/Data/valid"

classN_train_path = train_path + "/normal/"
classA_train_path = train_path + "/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/"
classL_train_path = train_path + "/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/"
classS_train_path = train_path + "/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa/"
classN_valid_path = val_path + "/normal/"
classA_valid_path = val_path + "/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/"
classL_valid_path = val_path + "/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/"
classS_valid_path = val_path + "/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa/"

In [ ]:
mean_sizes = []

print("CLASS: NORMAL")
mean_sizes.append(get_average_image_size(classN_train_path)[0])

print("\nCLASS: A")
mean_sizes.append(get_average_image_size(classA_train_path)[0])

print("\nCLASS: L")
mean_sizes.append(get_average_image_size(classL_train_path)[0])

print("\nCLASS: S")
mean_sizes.append(get_average_image_size(classS_train_path)[0])

In [ ]:

print("CLASS: NORMAL")
mean_sizes.append(get_average_image_size(classN_valid_path)[0])

print("\nCLASS: A")
mean_sizes.append(get_average_image_size(classA_valid_path)[0])

print("\nCLASS: L")
mean_sizes.append(get_average_image_size(classL_valid_path)[0])

print("\nCLASS: S")
mean_sizes.append(get_average_image_size(classS_valid_path)[0])

In [ ]:
mean_sizes = np.array(mean_sizes)
out_img_size = (int(np.round(mean_sizes[:, 0].mean())), int(np.round(mean_sizes[:, 1].mean())))

print(f"Mean of all images: {out_img_size}")

In [ ]:
out_img_dir = "/content/drive/MyDrive/W281/Final Project/Data_Cropped_and_Resized"
output_img_size = (256, 256)

crop_and_resize_images(train_path, output_img_size, out_img_dir)

In [ ]:
crop_and_resize_images(val_path, output_img_size, out_img_dir)


In [ ]:
img_name = "/content/drive/MyDrive/W281/Final Project/Data/train/normal/n8.jpg"
img = plt.imread(img_name)
plt.imshow(img)

In [ ]:
img_name = "/content/drive/MyDrive/W281/Final Project/Data_Resized/train/normal/n8.jpg"
img = plt.imread(img_name)
plt.imshow(img, cmap="gray")

In [ ]:
def show_slice_window(slice, level, window):
   """
   Function to display an image slice
   Input is a numpy 2D array
   """
   max = level + window/2
   min = level - window/2
   slice = slice.clip(min,max)

   fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20,10))
   ax[0].imshow(slice, cmap="gray")

   retval, thresh_gray = cv.threshold(slice,
                                      thresh=50,
                                      maxval=255,
                                      type=cv.THRESH_BINARY)
   ax[1].imshow(thresh_gray,
                cmap='gray',
                vmin=0,
                vmax=255)

   return thresh_gray

In [ ]:
img_N = cv.imread("/content/drive/MyDrive/W281/FinalProject/Data_Resized/train/normal/n9.jpg", cv.IMREAD_GRAYSCALE)
img_N = cv.equalizeHist(img_N)

# Calculate the histogram
hist, bins = np.histogram(img_N.flatten(), 256, [0, 256])

# Plot the histogram
plt.figure(figsize=(8, 6))
plt.bar(bins[:-1], hist, width=1, color='gray')
plt.title("Grayscale Histogram")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.show()

In [ ]:
  def get_PCA(X_list, n_components=2):
  pca_list = []
  xpca_list = []
  for X in X_list:
    pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X)
    X_pca = pca.transform(X)
    pca_list.append(pca)
    xpca_list.append(X_pca)
  return pca_list, xpca_list

def plot_PCA(X_list, labels, n_components=2):
  pca_list, xpca_list = get_PCA(X_list, n_components=n_components)

  plt.figure(figsize=(15,5))
  colors = ['b-', 'm-']
  for i in range(len(X_list)):
    plt.plot(np.cumsum(pca_list[i].explained_variance_ratio_), colors[i], label=labels[i])
  plt.xticks(np.arange(n_components)+1)
  plt.yticks(np.linspace(0, 1, 8))
  plt.grid(True)
  plt.xlabel('Number of components')
  plt.ylabel('Explained Variances')
  plt.legend()
  plt.show()

def get_tsne(X_list, n_components=2):
  xtsne_list = []
  for X in X_list:
    tsne = TSNE(n_components=n_components, random_state=0)
    X_tsne = tsne.fit_transform(X)
    xtsne_list.append(X_tsne)
  return xtsne_list

In [ ]:
labels = ['sobel edges', 'houndsfield edges']

training_features = [[img.flatten() for img in train_sobel_edges],
            [img.flatten() for img in train_hounsfield_edges]]

plot_PCA(training_features, labels, n_components=50)

LDA with Sobel Edges

In [ ]:
X_sobel_pca, X_hounsfield_pca = get_PCA(training_features, n_components=48)[-1]

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_sobel_pca, train_labels)

X_lda = lda.transform(X_sobel_pca)

coef_lda = lda.coef_[0]
intercept_lda = lda.intercept_[0]

plt.figure(figsize=(8, 6))

for label in np.unique(train_labels):
    plt.scatter(X_lda[train_labels == label, 0], X_lda[train_labels == label, 1], label=label)

line_x = np.array([X_lda[:, 0].min() - 1, X_lda[:, 0].max() + 1])
line_y = -(line_x * coef_lda[0] + intercept_lda) / coef_lda[1]

plt.plot(line_x, line_y, c='black', linewidth=2, label='Fitted Line')

plt.xlabel('LD1')
plt.ylabel('LD2')
plt.title('Fitted Line from Linear Discriminant Analysis')
plt.legend()
plt.grid(True)
plt.show()

LDA with Hounsfield Units

In [ ]:
lda.fit(X_hounsfield_pca, train_labels)

X_lda = lda.transform(X_hounsfield_pca)

coef_lda = lda.coef_[0]
intercept_lda = lda.intercept_[0]

plt.figure(figsize=(8, 6))

for label in np.unique(train_labels):
    plt.scatter(X_lda[train_labels == label, 0], X_lda[train_labels == label, 1], label=label)

line_x = np.array([X_lda[:, 0].min() - 1, X_lda[:, 0].max() + 1])
line_y = -(line_x * coef_lda[0] + intercept_lda) / coef_lda[1]

plt.plot(line_x, line_y, c='black', linewidth=2, label='Fitted Line')

plt.xlabel('LD1')
plt.ylabel('LD2')
plt.title('Fitted Line from Linear Discriminant Analysis')
plt.legend()
plt.grid(True)
plt.show()